# Mahalanobis Benchmark
Basic example of computing Mahalanobis distances for video clips with the TMCD algorithm.


In [ ]:
import os, zipfile, time, torch
from code.tmcd_fuctions_gpu import tmcd, _invSqrt, computeTensorMD

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
zip_path = '/content/tensor_dataset_sea.zip'  # upload zip here
assert os.path.exists(zip_path), 'tensor_dataset_sea.zip not found'
with zipfile.ZipFile(zip_path, 'r') as zf:
    zf.extractall('/content')
dataset = torch.load('/content/tensor_dataset_bigShip.pt', map_location='cpu')
X_full = (dataset.to(torch.float32) / 255.).to(device).contiguous()

# optional: drop every 4th frame
idx = torch.arange(X_full.size(-1), device=X_full.device)
mask = (idx % 4) != 0
X_full = X_full[..., mask].contiguous()

n, p1, p2, p3 = X_full.shape
print(f'Loaded: {n} clips  ({p1}×{p2}×{p3})')


In [ ]:
start = time.perf_counter()
tmcd_res = tmcd(X_full, alpha=0.6, nSubsets=100, nBest=10,
                 maxIterCshort=2, maxIterFFshort=2,
                 maxIterCfull=20, maxIterFFfull=50,
                 tolC=1e-4, tolFF=1e-3, beta=0.9999)
if torch.cuda.is_available():
    torch.cuda.synchronize()
elapsed = time.perf_counter() - start
print(f'TMCD finished in {elapsed:.1f}s')
print('Good clips:', tmcd_res['finalGood'].numel(), '/', n)


In [ ]:
invSqrt1 = _invSqrt(tmcd_res['Sigma1'])
invSqrt2 = _invSqrt(tmcd_res['Sigma2'])
invSqrt3 = _invSqrt(tmcd_res['Sigma3'])
TMDsq_all = computeTensorMD(X_full, invSqrt1, invSqrt2, invSqrt3)
top_md2, top_idx = torch.topk(TMDsq_all.cpu(), k=5, largest=True)
print('Top clips:', list(zip(top_idx.tolist(), top_md2.tolist())))
